In [4]:
!pip install pandas==1.3.2

  Using cached pandas-1.3.2-cp37-cp37m-win_amd64.whl (10.0 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.0
    Uninstalling pandas-1.3.0:
      Successfully uninstalled pandas-1.3.0


In [1]:
!pip install swifter
!pip install Sastrawi

# Import Library

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import string
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
import csv
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Kanzul
[nltk_data]     Faisal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Kanzul
[nltk_data]     Faisal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Kanzul Faisal\Documents\Proposal PI\Komentar_Youtube.csv')
df.head()

ImportError: cannot import name 'DtypeArg' from 'pandas._typing' (C:\Users\Kanzul Faisal\Anaconda\lib\site-packages\pandas\_typing.py)

In [ ]:
del df['Unnamed: 0']
df.head()

In [ ]:
#cleaning Text
def remove_punct(text):
    text = re.sub(r'[^a-zA-Z0-9]', ' ', str(text))
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = re.sub(r'\b\w(1,2)\b', '', text)
    text = re.sub(r'\s\s+', ' ', text)
    text = re.sub(r"\d+", "", text)
    return text
df['Komentar'] = df['Komentar'].apply(remove_punct) 

In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
df['Komentar'] = df['Komentar'].str.lower()
df.head()

In [ ]:
def replace_repeat_character(text):
    # Pattern to look for three or more repetitions of any character, including newlines (contoh goool -> gol).
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", text)

df['Komentar'] = df['Komentar'].apply(replace_repeat_character) 
df.head()

In [ ]:
from nltk.tokenize import word_tokenize

def tokenize(text):
    return nltk.word_tokenize(text)
    
df['TOKENIZATION'] = df['Komentar'].apply(tokenize)
df.head(10)

In [ ]:
katabaku = csv.reader(
	open(r"C:\Users\Kanzul Faisal\Documents\Proposal PI\Project PI\kamus_alay_referensi_2.csv"), delimiter=";") # ambil file csv kata baku menjadi array

kamus_katabaku = {} # empty dictionary untuk kamus kata baku
try:
    for row in katabaku : # membuat kamus kata baku dengan input kata tidak baku dan output kata bakunya
        kamus_katabaku[row[0]] = row[1]
except:
    pass

In [ ]:
print(kamus_katabaku)

In [ ]:
def normalized_term(text):
    return [kamus_katabaku[term] if term in kamus_katabaku else term for term in text]
df['TOKENIZATION'] = df['TOKENIZATION'].apply(normalized_term) 
df.head(10)

In [ ]:
stopword =nltk.corpus.stopwords.words('indonesian')
stopword.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                 'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                 'krn', 'nya', 'nih', 'sih', 
                 'si', 'tau', 'tuh', 'utk', 'ya','sdh',
                 'aja', 'n', 't', 'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                 '&amp', 'yah', 'njir', 'wkwk', 'haha', 'hehe', 'rs', 'yaa',
                 'nii', 'bin', 'lahh', 'hmm', 'lieur', 'njarem', 'kale',
                 'hd', 'an', 'ob', 'bro', 'ud', 'iyaa', 'guys', 'doang',
                 'doank', 'eh', 'kek', 'or', 'kah', 'eso', 'tuh', 'dh', 'dnk',
                 'mh', 'ja', 'tu', 'anu', 'ni', 'ku', 'blz', 'gy', 'skg', 'eeh',
                 'pas','tar','bc','x','dong','d'])

In [ ]:
word_list=['enggak','tidak','jangan','belum']
stopword = [word for word in stopword if word not in word_list] #make a copy of the word_list

In [ ]:
def remove_stopwords(text):
    return [word for word in text if word not in stopword]
    
df['STOP_REMOVAL'] = df['TOKENIZATION'].apply(remove_stopwords)
df.head(10)

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
import swifter

# implementasi pada data kita

# stemmed
def stemmed_wrapper(text):
    return stemmer.stem(text)

text_dict = {}

for document in df['STOP_REMOVAL']:
    for text in document:
        if text not in text_dict:
            text_dict[text] = ' '
            
print(len(text_dict))
print("------------------------")

for text in text_dict:
    text_dict[text] = stemmed_wrapper(text)
    print(text,":" ,text_dict[text])
    
print(text_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [text_dict[text] for text in document]


In [ ]:
df['STEMMER'] = df['STOP_REMOVAL'].apply(get_stemmed_term)
df.head(10)

In [ ]:
def fit_stemmer(text):
     text = np.array(text)
     text = ' '.join(text)
     return text

df['Komentar_Final'] = df['STEMMER'].apply(lambda x: fit_stemmer(x))
df.head(10)

In [ ]:
df.to_csv(r'C:\Users\Kanzul Faisal\Documents\Proposal PI\Project PI\PreProcessing.csv')

In [ ]:
import pickle
pickle.dump(stopword,
            open('stopword.pkl', 'wb'),
            protocol=4)

In [ ]:
import pickle
pickle.dump(kamus_katabaku,
            open('kamus_katabaku.pkl', 'wb'),
            protocol=4)